This notebook loads all the igrams, references them to some region/group of pixels, and plots all of them.

In [ ]:
# import stuff
import os,sys
import glob
from osgeo import gdal
import datetime 
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib  widget

Load in all unfilt, unwrapped igrams.

In [ ]:
# igramsDir is where the cropped, unwrapped igrams are 
igramsDir = '/data/ocp4/LagunaSalada/cropped_9000_1800_18000_3600/cropped_northern_shore_LS_for_unwrapping/unfilt_unwrapped/'
# this one is to pick out the dates 
datesDir = '/data/ocp4/LagunaSalada/cropped_9000_1800_18000_3600/'
# shape of unwrapped igrams
dx = 3000
dy = 750
# load in dates, make sure they are in order
dates = [os.path.basename(x) for x in glob.glob(datesDir+"SLC_vh/2*")]; dates = sorted(dates)
nd = len(dates); numIgrams = nd-1
# set the driver first, only do once.
driver=gdal.GetDriverByName('ISCE')
unwr_full_res = np.ndarray((numIgrams,dy,dx),dtype='float')
# loop through the dates 
for i in range(numIgrams): # numIgrams if doing all of them at once
    date1=dates[i]
    date2=dates[i+1]
    igramFile = igramsDir+date1+'_'+date2+'_unfilt.unw'
    ds = gdal.Open(igramFile,gdal.GA_ReadOnly)
    unwr_full_res[i] = ds.GetRasterBand(1).ReadAsArray(0,0,dx,dy)
print('unfiltered unwrapped igrams loaded.')

Reference all igrams to reference region/box/group of pixels. <br>
For now, use the initial box. 

In [ ]:
%%script echo skipping 
# correct all igrams so there's a zero/stationary point 
# initialize an array for corrected igrams 
unwr_full_res_cor_2 = np.ndarray(np.shape(unwr_full_res),dtype='float') # this is two reference to a smaller region and compare 
for i in range(numIgrams):
    # remove the average phase from small reference region at foot of mountains   
    unwr_full_res_cor_2[i] = unwr_full_res[i]-np.mean(unwr_full_res[i,150:180,800:900])
print('igrams corrected by subtracting avg value over small reference region in mountains.')
# del unwr_full_res

Load the reference pixel mask to use as reference.

In [ ]:
# load reference pixels mask 
maskFile = '/data/ocp4/LagunaSalada/cropped_9000_1800_18000_3600/cropped_northern_shore_LS_for_unwrapping/masks/ref_pixels_mask.r4'
ds = gdal.Open(maskFile,gdal.GA_ReadOnly)
ref_mask = ds.GetRasterBand(1).ReadAsArray(0,0,dx,dy)
print('Reference pixel mask loaded.')
# plot it to make sure I read it in correctly
plt.figure(figsize=(5,4.5))
plt.imshow(ref_mask, cmap='gray',aspect='auto',interpolation='nearest',vmin=0,vmax=1.0)
plt.xlim(dx,0); plt.xticks([]); plt.yticks([])
plt.xlabel('range'); plt.ylabel('azimuth'); plt.title('reference pixels mask') 
plt.colorbar(); plt.show()

Reference igrams to ref pixels above.

In [ ]:
# correct all igrams so there's a zero/stationary point  
mask_y,mask_x = np.nonzero(ref_mask)
unwr_full_res_cor = np.ndarray(np.shape(unwr_full_res),dtype='float') # this is two reference to a smaller region and compare 
for i in range(numIgrams):
    # remove the average phase from small reference region at foot of mountains   
    unwr_full_res_cor[i] = unwr_full_res[i]-np.mean(unwr_full_res[i,mask_y,mask_x])
print('igrams corrected by subtracting avg value of ref pixels.')
# del unwr_full_res

Get precip data.

In [ ]:
# Read in 2day precipitation data
precip = np.loadtxt('/data/ocp4/LagunaSalada/3_precip/precip_2days.txt', dtype='float')
#precip = np.loadtxt('/data/ocp4/LagunaSalada/3_precip/precip_cumulative.txt', dtype='float')
print(np.shape(precip))

Plot them all. There are 210. This takes a while.

In [ ]:
rows = 105 # 105 for all, or int(numIgrams/2)
cols = 2
figWid = 10
figHgt = 4.5*rows
plt.figure(figsize=(figWid,figHgt),tight_layout=True)
vmin = 0
vmax = 20

for i in range(numIgrams):  # for all, use range(numIgrams)
    date1 = dates[i]; date2 = dates[i+1]
    plt.subplot(rows,cols,i+1)
    plt.imshow(unwr_full_res_cor[i],aspect='auto',interpolation='nearest')
    plt.colorbar(orientation='vertical',pad=0.03,label='phase (rad)'); plt.set_cmap('jet')
    plt.xlim([dx,0]); plt.ylim([dy,0])
    plt.xticks([]); plt.yticks([]) #plt.xlabel('range'); plt.ylabel('azimuth')
    plt.title(str(date1)+', '+str(round(precip[i],2))+' cm precip\n'+str(date2)+', '+str(round(precip[i+1],2))+' cm precip'+'\n i = '+str(i))

Just plot igrams from the storm at the end of 2018

In [ ]:
#vmin = 0
#vmax = 20
for i in np.arange(45,50):  # for all, use range(numIgrams)
    date1 = dates[i]; date2 = dates[i+1]
    plt.figure(figsize=(5,4.5),tight_layout=True)
    plt.imshow(unwr_full_res_cor[i],aspect='auto',interpolation='nearest')
    plt.colorbar(orientation='vertical',pad=0.03,label='phase (rad)'); plt.set_cmap('jet')
    plt.xlim([dx,0]); plt.ylim([dy,0])
    plt.xticks([]); plt.yticks([]) #plt.xlabel('range'); plt.ylabel('azimuth')
    plt.title(str(date1)+', '+str(round(precip[i],2))+' cm precip\n'+str(date2)+', '+str(round(precip[i+1],2))+' cm precip'+'\n i = '+str(i))

Load in coherence files created with notebook /data/ocp4/LagunaSalada/2_coherence/produce_coherence_files_for_sequential_igrams.ipynb 
<br> This takes a few minutes. 

In [ ]:
# read in coherence files
saveDir = '/data/ocp4/LagunaSalada/2_coherence/coh_tip_LS_all_igrams/'
coh = np.ndarray((np.shape(unwr_full_res)),dtype='float')
for i in range(numIgrams): #range(numIgrams)
    date1=dates[i]
    date2=dates[i+1]
    file_name = 'coh_'+str(date1)+'_'+str(date2)+'.r4'
    f1 = gdal.Open(saveDir+file_name, gdal.GA_ReadOnly)
    coh[i] = f1.ReadAsArray()
f1 = None
print('Coh maps loaded.')

Plot coherence maps for the rainy dates above.

In [ ]:
for i in np.arange(45,50):  # for all, use range(numIgrams)
    date1 = dates[i]; date2 = dates[i+1]
    plt.figure(figsize=(5,4.5),tight_layout=True)
    plt.imshow(coh[i],aspect='auto',interpolation='nearest',extent=[0,dx,dy,0],vmin=0,vmax=1.0)
    plt.colorbar(orientation='vertical',pad=0.03,label='phase (rad)'); plt.set_cmap('jet')
    plt.xlim([dx,0]); plt.ylim([dy,0])
    plt.xticks([]); plt.yticks([]) #plt.xlabel('range'); plt.ylabel('azimuth')
    plt.title(str(date1)+', '+str(round(precip[i],2))+' cm precip\n'+str(date2)+', '+str(round(precip[i+1],2))+' cm precip'+'\n i = '+str(i))